In [1]:
ENV["GKS_ENCODING"] = "utf-8"
using NLsolve
using Plots
using SharedArrays


using HDF5
using LadderDGA
using Suppressor
using Logging
using Pkg
Pkg.activate("/home/julian/Hamburg/LadderDGA.jl")
using LadderDGA

cfg_file = "/home/julian/Stuttgart/error_bz/config.toml"

io = stdout
metafmt(level::Logging.LogLevel, _module, group, id, file, line) = Logging.default_metafmt(level, nothing, group,     id, nothing, nothing)
logger = ConsoleLogger(io, Logging.Info, meta_formatter=metafmt, show_limited=true, right_justify=0)
global_logger(logger);

  Activating project at `~/Hamburg/LadderDGA.jl`


In [2]:
mP, sP, env, kGrids, qGridLoc, freqList = readConfig(cfg_file)
kG = kGrids[1]
νGrid, sumHelper_f, impQ_sp, impQ_ch, GImp, GLoc, GLoc_fft, Σ_loc, FUpDo  = setup_LDGA(kG, freqList, mP, sP, env);

[ Info: Reading Inputs...
[ Warning: Could not find E_kin_DMFT, E_pot_DMFT key in input
[ Info: setting usable ranges of sp and ch channel from 62:98 and 62:98 to the same range of 62:98
┌ Info: Inputs Read. Starting Computation.
│ Local susceptibilities with ranges are:
│ χLoc_sp(62:98) = 0.3003, χLoc_ch(62:98) = 0.0917 
└ sum χupup check (fit, tail sub, tail sub + fit, expected): 0.19596777560898987 ?≈? 0.1959677756089899 ?=? 0.1959677756089899 ?≈? 0.25"


In [3]:
# "local" correction quantities
bubbleLoc = calc_bubble(νGrid, GImp_fft, qGridLoc, mP, sP);
locQ_sp = calc_χ_trilex(impQ_sp.Γ, bubbleLoc, qGridLoc, νGrid, sumHelper_f, mP.U, mP, sP);
locQ_ch = calc_χ_trilex(impQ_ch.Γ, bubbleLoc, qGridLoc, νGrid, sumHelper_f, -mP.U, mP, sP);
Σ_ladderLoc = calc_Σ(locQ_sp, locQ_ch, bubbleLoc, GImp_fft, FUpDo,
                     qGridLoc, sumHelper_f, mP, sP)

Σ_ladderLoc = Σ_ladderLoc .+ mP.n * mP.U/2.0;
println("non local")
# ladder quantities
bubble = calc_bubble(νGrid, GLoc_fft, kG, mP, sP);
nlQ_sp = calc_χ_trilex(impQ_sp.Γ, bubble, kG, νGrid, sumHelper_f, mP.U, mP, sP);
nlQ_ch = calc_χ_trilex(impQ_ch.Γ, bubble, kG, νGrid, sumHelper_f, -mP.U, mP, sP);

nlQ_sp_nλ = deepcopy(nlQ_sp);
nlQ_ch_nλ = deepcopy(nlQ_ch);

λ_sp, λ_spch  = λ_correction!(impQ_sp, impQ_ch, FUpDo, Σ_loc, Σ_ladderLoc, nlQ_sp, nlQ_ch, bubble, GLoc_fft, kG, mP, sP)

Σ_ladder = calc_Σ(nlQ_sp, nlQ_ch, bubble, GLoc_fft, FUpDo, kG, sumHelper_f, mP, sP)
Σ_ladder_corrected = Σ_ladder .- Σ_ladderLoc .+ Σ_loc[1:size(Σ_ladder,1)];

LoadError: UndefVarError: GImp_fft not defined

In [4]:
bs = sum(bubble, dims=3)[:,:,1];

LoadError: UndefVarError: bubble not defined

In [5]:
heatmap(real(reshape(bs[1,:],(10,10))))

LoadError: UndefVarError: bs not defined

In [6]:
heatmap(real(reshape(kG.ϵkGrid,(10,10))))

LoadError: DimensionMismatch("new dimensions (10, 10) must be consistent with array size 400")